## Model Success Probability Measure

The most intuitive question one can address is on the success rate of the model in generating the correct answer to the prompt. We are able to measure this probability by comparing the output with the truth data. We draw random samples of articles with their truth data from kaggle and run the LLM on each article independently. We then use various distance measures to qunatify the similarity between the output and the truth data. We finally put a threshold level to the results such that if the measurement outcome deviates from this threshold, then we consider the model failed at generating correct outputs. Therefore it is important to know what the threshold should be. The most restrictive threshold will be zero, showing that there is no difference between the outcome and the truth. This is reasonable for research question 1 and 2, however not a good approach for research question 3 because the summary output from the model will most likely have some difference compared to the original abstract of the article. For research question 3, one can choose an reasonable threshold for analysis, but the best approach should be based on the interpretation of a human after having read the summary output from the model and compared it with the abstract.

Once the appropriate measurement strategy is determined, one can conclude whether the model has successfully generated the correct answer to the prompt or not. The result forms a random sample of Bernoulli trials. Suppose $X_1,...,X_n$ are the random sample of the experimental result for a large language model. The Bernoulli random variables have a support on $\{0, 1\}$, with $X_i=1$ if the model generates a correct answer to the prompt with probability $p$, otherwise $X_i=0$ with probability $1-p$. The success rate $p$ is the parameter of the Bernoulli distribution. Our goal is therefore to estimate this success probabilty and compare it between different models.


We can have various hypothesis tests on the success probability $p$. One may consider a right-tailed hypothesis:

$$
H_0: ~ p \leq p_0 \\
H_1: ~ p> p_0
$$

such that we would like to know if the success probability is greater than certain value $p_0$. One may also consider a two-sample hypothesis test:
$$
H_0: ~ p_1 \leq p_2 \\
H_1: ~ p_1 > p_2
$$

such that we would like to know if the success probability of model 1 $p_1$ is greater than the one of model 2 $p_2$. Note that since $p$ is a real number between 0 and 1, it is not informative to havae a two-sided test $H_0: ~ p=0$ VS $H_a: p \neq 0$, so we omit such test.

A general test statistic one can find in most of Statistics textbooks is the normal approximation to the binomial distribution. We can use the likelihood ratio to derive the test statistic for our composite hypothesis:
$$
\Lambda(\vec{x})=\frac{\sup_{p \leq p_0} p_0^{\sum_{i=1}^{n} x_i} (1-p_0)^{n-\sum_{i=1}^{n} x_i} }{\sup_{p \in \Theta}  p^{\sum_{i=1}^{n} x_i} (1-p)^{n-\sum_{i=1}^{n} x_i}}
$$

The maximum likelihood estimator (MLE) of $p$ is $\hat{p}=\frac{1}{n}\sum_{i=1}^{n} x_i$, so $\sum_{i=1}^{n} x_i = n\hat{p}$. The LRT statistic becomes:
$$
\Lambda(\vec{x})=
\begin{cases}
1 & p_0 \geq \frac{y}{n}\\
\frac{(p_0)^{n\hat{p}} \cdot (1-p_0)^{n(1-\hat{p})} }{(\hat{p})^{n\hat{p}} \cdot (1-\hat{p})^{n(1-\hat{p})}} & p_0 < \frac{y}{n}
\end{cases}
$$

Let $g(\vec{x}):=-\ln{\Lambda(\vec{x})} = n\hat{p}(\ln{\hat{p}}-\ln{p_0})+n(1-\hat{p})(\ln{(1-\hat{p})} - \ln{(1-p_0)})$.

By Taylor's expansion, $\ln{(1-\hat{p})} - \ln{(1-p_0)} \approx - \frac{\hat{p}-p_0}{1-p_0}$, $\ln{\hat{p}} - \ln{p_0} \approx \frac{\hat{p}-p_0}{p_0}$. Therefore,
$$
g(\vec{x}) = n(\hat{p}-p_0)\left( \frac{\hat{p}-p_0}{p_0(1-p_0)} \right) = \frac{(\hat{p}-p_0)^2}{p_0(1-p_0)/n}.
$$

By Central Limit Theorem, $T(\vec{x}):=\frac{\hat{p}-p_0}{\sqrt{p_0(1-p_0)/n}}$ is Normal(0,1) distributed. Thus we have that
$$
\Lambda(\vec{x}) < \lambda_0 \equiv T(\vec{x}) > \lambda_0
$$

Our hypothesis test is
$$
\phi(\vec{x}) = 
\begin{cases}
1, \quad T(\vec{x}) > \lambda_0\\
0, \quad T(\vec{x}) \leq \lambda_0
\end{cases}
$$

Using the pivotal quantity $Q(\vec{x}, p):=\frac{\hat{p}-p}{\sqrt{p(1-p)/n}}$ we can get the ($1-\alpha$)-level confidence interval for the success rate as $[L(\vec{X}), R(\vec{X})]$ such that:
$$
\mathbb{P}(p \in [L(\vec{X}), R(\vec{X})])\geq 1-\alpha
$$

The interpretation of the confidence interval is important. In a frequentist point of view, our unknown parameter $p$ is a true but fixed value. The confidence interval on the other hand is a random set, thus it is incorrect to state that we are 99% sure that the confidence interval covers the unknown true paramter. It is also incorrect to say that the unknown parameter falls within the confidence interval 99% of the time, because in this statement the unknown parameter is falsely treated as a random variable. Therefore the appropriate interpretation of the confidence interval is: when the experiment is repeated many times to generate random samples and their perspective confidence intervals, 99% of these confidence intervals will cover the unknown fixed parameter.

Alternatively, we can use the Bayesian approach to gain a different understanding of our unknown parameter and to draw inference on it. Assuming we have a prior knowledge of this success rate, say 80%, before observing the data. The Baysian approach allows us to "correct" our degree of beliefs based on the actual observation. In particular, the Bayes' Theorem says that 
$$
P(A|B) = \frac{P(A) \cdot P(B|A)}{P(B)}
$$
where $P(A)$ is the prior probability reflecting our prior belief and $P(A|B)$ is the posterior probability after having observed the event B. Such analysis allows us to make an inference on the success rate in a very different point of view, which we will explain later.

Suppose we assume $Beta(\alpha, \beta)$ is the prior distribution of the success rate p. The prior density function is:
$$
f(p; \alpha,\beta) = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} p^{\alpha-1} (1-p)^{\beta-1}, ~ p\in (0,1)
$$
We know the expectation and the variance of Beta distribution as:
$$
\begin{gather*}
E(p)= \frac{\alpha}{\alpha+\beta}\\
Var(p)= \frac{\alpha \beta}{(\alpha+\beta)^2(\alpha+\beta+1)}
\end{gather*}
$$
Thus if we assume that the average success rate is $t_1$ with a variance of $t_2$, then we can solve for the parameters of the prior distribution as 
$$
\alpha = - \frac{t_1}{t_2}\cdot (t_1^2-t_1+t_2), \quad \beta=\alpha \cdot \frac{1-t_1}{t_1}.
$$ 

Since the random sample $X_1,...,X_n$ follows Bernoulli(p) distribution, we have a conjugate pair of the prior and posterior distributions:
$$
\begin{align*}
f(p|\vec{x})&=\frac{f(p)f(\vec{x}|p)}{\int_0^1 f(p)f(\vec{x}|p) dp}=\frac{\frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} p^{\alpha-1+\sum_{i=1}^n x_i} (1-p)^{\beta-1+n-\sum_{i=1}^n x_i}}{\int_0^1 \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} p^{\alpha-1+\sum_{i=1}^n x_i} (1-p)^{\beta-1+n-\sum_{i=1}^n x_i} dp}=\frac{p^{\alpha-1+\sum_{i=1}^n x_i} (1-p)^{\beta-1+n-\sum_{i=1}^n x_i}}{\frac{\Gamma(\alpha+\sum_{i=1}^n x_i)\Gamma(\beta+n-\sum_{i=1}^n x_i)}{\Gamma(\alpha+\beta+n)}}\\
&=\frac{\Gamma(\alpha+\beta+n)}{\Gamma(\alpha+\sum_{i=1}^n x_i)\Gamma(\beta+n-\sum_{i=1}^n x_i)} p^{\alpha-1+\sum_{i=1}^n x_i} (1-p)^{\beta-1+n-\sum_{i=1}^n x_i}
\end{align*}
$$
i.e. the posterior distribution is $Beta(\alpha+\sum_{i=1}^n x_i, \beta+n-\sum_{i=1}^n x_i)$ with the mean and variance:
$$
\begin{gather*}
E(p|\vec{x})=\frac{\alpha+\sum_{i=1}^n x_i}{\alpha+\beta+n}\\
Var(p|\vec{x})=\frac{(\alpha+\sum_{i=1}^n x_i)(\beta+n-\sum_{i=1}^n x_i)}{(\alpha+\beta+n)^2(\alpha+\beta+n+1)}
\end{gather*}
$$
One can see that asymptotically when we have a large sample size n, the mean of the posterior distribution approaches to the maximum likelihood estimator $\hat{p} =\frac{1}{n}\sum_{i=1}^n x_i$. Thus the specification of a prior is less important if the sample size is large.  This allows us to make a subjective assumption of the success rate based on a probability distribution instead of viewing it as a fixed but unknown number in the frequentist hypothesis test. In the case where $\alpha=\beta=1$, we obtain the non-informative prior $Beta(1,1)$, which is $Uniform([0,1])$ distribution. It's called non-informative because we assume that all p are equally probable. In other words, we no longer put a subjective assumption on $p$. 

After having obtained the posterior distribution, we can get the ($1-\alpha$)-level credible interval for the success rate based on the quantile of the posterior as $[L(\vec{x}), R(\vec{x})]$ such that:
$$
\mathbb{P}(\bold{p} \in [L(\vec{x}), R(\vec{x})] |\vec{x}) = \int_{L(\vec{x})}^{R(\vec{x})} f(p|\vec{x})dp = 1-\alpha
$$
With such definition, it is perfectly fine to interpret the credible interval as: the probability that our success rate falls within the given credible interval is (1-$\alpha$)%. 

Various research papers have shown that the Bayesian Uniform prior yields better estimation of the confidence interval than the one obtained from the normal approximation of the binomial distribution. We can make a simulation to justify this.

One can also test a two-sample hypothesis: $H_0: p1 \leq p2$ VS. $H_1: p1 > p2$ where $p_1$ is the parameter of Model1, and $p_2$ is the parameter of Model2. In the frequentist approach, we often need to specify the significance level $\alpha$, which is the type I error defined by:
$$
\alpha = E[\phi(\vec{X}) | H_0] = \mathbb{P}(T(\vec{X}) \in R) = \int_R f_T(t)dt
$$
As we saw earlier, this requires us to find the distribution of the statistic $T(\vec{X})$ under the null hypothesis and its corresponding rejection region $R$, which in general is difficult. However, with Bayesian analysis one can evaluate directly the probability of parameter $p_1$ greater than parameter $p_2$. We can do this by using the Monte Carlo method to evaluate the following integral:
$$
P(p_1 > p_2 |\vec{x}) = \int_0^1 \int_0^1 \mathbb{I}(p_1 > p_2 |\vec{x}) \cdot f(p_1|\vec{x})f(p_2|\vec{x})dp_1 dp_2 = E[\mathbb{I}(p_1 > p_2 |\vec{x})]
$$
where $\mathbb{I}(p_1 \leq p_2|\vec{x})$ is the indicator function given the data and $f(p_1|\vec{x})$, $f(p_2|\vec{x})$ are the posterior Beta probability density functions for Model1 and Model2 respectively. To estimate this probability, one only needs to simulate pairs of Beta random variables from the two posterior distributions simultaneously, and calculate the average of the indicator function. 



In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import plotly.express as px
from ModelEvaluation import Bayes


### Section 1: Show the Superiority of Bayesian Inference

In [2]:
def effCP(n, p, alpha, beta, alphalevel=0.05):
    k=np.linspace(0,n,n+1)
    param_1 = (alpha+np.array(k)).tolist()
    param_2 = (beta+n-np.array(k)).tolist()
    #DataF=pd.DataFrame({'k':k, 'param_1':param_1, 'param_2': param_2})
    
    CI_Wald=[[x/n + stats.norm.ppf(alphalevel/2)*math.sqrt((x/n)*(1-x/n)/n), x/n - stats.norm.ppf(alphalevel/2)*math.sqrt((x/n)*(1-x/n)/n)] for x in k]
    CI_Bayes=[stats.beta.ppf([alphalevel/2, 1-alphalevel/2], x,y) for x, y in zip(param_1, param_2)]
    
    Covered_Wald=[1 if p >= x[0] and p <= x[1] else 0 for x in CI_Wald]
    Covered_Bayes=[1 if p >= x[0] and p <= x[1] else 0 for x in CI_Bayes]

    CP_Wald=Covered_Wald*stats.binom.pmf(k, n, p)
    CP_Bayes=Covered_Bayes*stats.binom.pmf(k, n, p)
    return([sum(CP_Wald), sum(CP_Bayes)])


def MeanEffCov(n, p, alpha, beta, alphalevel=0.05):
    effData=[]
    for i in p:
        effData.append(effCP(n, i, alpha, beta, alphalevel))
    effData=pd.DataFrame(effData,columns=['CP_Wald','CP_Bayes'])
    return([effData['CP_Wald'].mean(), effData['CP_Bayes'].mean()])


def CI_z(data, threshold, alphalevel):
    n=len(data)
    p_hat=len(data[data<=threshold])/n
    CI=[p_hat + stats.norm.ppf(alphalevel/2)*math.sqrt(p_hat*(1-p_hat)/n), p_hat - stats.norm.ppf(alphalevel/2)*math.sqrt(p_hat*(1-p_hat)/n)]
    CI=["{:.2%}".format(i) for i in CI]
    print("The maximum likelihood estimate is: " + str("{:.2%}".format(p_hat)))
    print(f"The 99%-level confidence interval for {data.name} distance measure is: " + str(CI))

In [3]:
n=15
p=np.linspace(0.025,0.975,1000)
effData=[]
for proportion in p:
    effData.append(effCP(n, proportion, 1, 1, 0.05))
effData=pd.DataFrame(effData,columns=['CP_Wald','CP_Bayes'])
effData.insert(0,'p', p)
effData

,p,CP_Wald,CP_Bayes
0,0.025000,0.315552,0.947106
1,0.025951,0.325426,0.943468
2,0.026902,0.335159,0.939743
3,0.027853,0.344752,0.935932
4,0.028804,0.354205,0.932038
...,...,...,...
995,0.971196,0.354205,0.932038
996,0.972147,0.344752,0.935932
997,0.973098,0.335159,0.939743
998,0.974049,0.325426,0.943468


In [4]:
n=list(range(10,350,10))
p=np.linspace(0.01,0.99,200)
alpha=1
beta=1
alphalevel=0.05

MeanEffData=[]
for size in n:
    MeanEffData.append(MeanEffCov(size, p, alpha, beta, alphalevel))

MeanEffData=pd.DataFrame(MeanEffData,columns=['CP_Wald','CP_Bayes'])
MeanEffData.insert(0,'n', n)
MeanEffData=MeanEffData.reset_index()
MeanEffData=pd.melt(MeanEffData, id_vars='n', value_vars=['CP_Wald', 'CP_Bayes'])

In [5]:
fig=px.line(MeanEffData, x='n', y='value', color='variable',
            labels=dict(n="Sample Size", value="Coverage Probability", variable="Method"))
arr=np.arange(0, 1, 0.01)
fig.update_layout(yaxis=dict(tickmode='array', tickvals=arr, ticktext=[str(round(x,4)) for x in arr]),
                  xaxis=dict(tickmode='linear', tick0=0, dtick=20),
                  width=1200, height=700,
                  title=dict(text=f"Mean Coverage Probability for 95% Confidence Level", x=0.5)
                  )
fig.add_hline(y=0.95,line_dash="dash", line_color="black", line_width=0.5)
fig.show()

### Section 2: Show a Simulation with Bayesian Inference

In [3]:
x=[(f"sim_p:{p}", stats.bernoulli.rvs(p, size=60)) for p in [0.8342,0.21,0.72,0.97,0.05,0.4]]
x=dict(x)

threshold=[[1]*6]
alphalevel=0.05
alpha=[1]*6
beta=[1]*6

result=Bayes(x, threshold, alpha, beta, alphalevel, direction='greater')
display(result.res)
result.Plot_Bayes(height=500)

,Component,Expected Success Probability,Variance of Success Probability,95.0%-level Credible Interval,Threshold
0,sim_p:0.8342,0.7903,0.002630,"[0.6816, 0.8814]","[1, 1, 1, 1, 1, 1]"
1,sim_p:0.21,0.1935,0.002478,"[0.106, 0.2998]","[1, 1, 1, 1, 1, 1]"
2,sim_p:0.72,0.6613,0.003555,"[0.54, 0.7727]","[1, 1, 1, 1, 1, 1]"
3,sim_p:0.97,0.9516,0.000731,"[0.8865, 0.9897]","[1, 1, 1, 1, 1, 1]"
4,sim_p:0.05,0.0968,0.001387,"[0.037, 0.181]","[1, 1, 1, 1, 1, 1]"
5,sim_p:0.4,0.4355,0.003902,"[0.3155, 0.5594]","[1, 1, 1, 1, 1, 1]"


In [4]:
threshold=[[1]*6]
alphalevel=0.05
alpha=[1,1,1,1,   9429/800,  12] #the customized prior has mean 0.035 and std 0.01 for the 5th p, and 0.5, 0.2 for the 6th p
beta=[1,1,1,1,   259971/800,  12]

result=Bayes(x, threshold, alpha, beta, alphalevel, direction='greater')
display(result.res)
result.Plot_Bayes(height=500)

,Component,Expected Success Probability,Variance of Success Probability,95.0%-level Credible Interval,Threshold
0,sim_p:0.8342,0.7903,0.002630,"[0.6816, 0.8814]","[1, 1, 1, 1, 1, 1]"
1,sim_p:0.21,0.1935,0.002478,"[0.106, 0.2998]","[1, 1, 1, 1, 1, 1]"
2,sim_p:0.72,0.6613,0.003555,"[0.54, 0.7727]","[1, 1, 1, 1, 1, 1]"
3,sim_p:0.97,0.9516,0.000731,"[0.8865, 0.9897]","[1, 1, 1, 1, 1, 1]"
4,sim_p:0.05,0.0423,0.000102,"[0.0248, 0.0642]","[1, 1, 1, 1, 1, 1]"
5,sim_p:0.4,0.4524,0.002914,"[0.3479, 0.559]","[1, 1, 1, 1, 1, 1]"


### Section 3: Implementation of Bayesian Inference on Real Data

In [2]:
import ast
df1 = pd.read_csv("rq1_results.csv")
df1=df1.replace([np.inf,-np.inf], np.nan)
df1=df1.dropna()
print("Length of data: " + str(len(df1['id'])))
display(df1)

df2 = pd.read_csv("rq2_results.csv")
df2=df2.replace([np.inf,-np.inf], np.nan)
df2=df2.dropna()
df2['Cosim_Values']=df2['Cosim_Values'].apply(ast.literal_eval)
print("Length of data: " + str(len(df2['Cosim_Values'][0])))
display(df2)

df1_gpt4 = pd.read_csv("rq1_results_gpt4.csv")
df1_gpt4=df1_gpt4.replace([np.inf,-np.inf], np.nan)
df1_gpt4=df1_gpt4.dropna()
print("Length of data: " + str(len(df1_gpt4['id'])))
display(df1_gpt4)

Length of data: 7224


,id,title_levenshtein,abstract_levenshtein,authors_levenshtein
0,2305.00604,0.0,0.0,22.285714
2,2305.00607,0.0,0.0,16.666667
4,2305.00617,0.0,141.0,25.000000
5,2305.00618,0.0,0.0,22.333333
7,2305.00620,0.0,692.0,14.800000
...,...,...,...,...
9942,2306.05419,0.0,746.0,27.000000
9943,2306.05421,0.0,521.0,17.333333
9944,2306.05423,0.0,0.0,16.200000
9946,2306.05427,0.0,422.0,18.000000


Length of data: 1726


,Name,Mean,Count,Cosim_Values
0,mathematics,0.528710,1726,"[0.4215032589683878, 0.39318921606835533, 0.0,..."
1,economics,0.538987,110,"[0.7826176108015402, 0.0, 0.6252434790548467, ..."
2,quantitative_biology,0.539134,195,"[0.5205144185370245, 0.3607601419645778, 0.532..."
3,statistics,0.543196,603,"[0.4215032589683878, 0.6845831067637522, 0.488..."
4,computer_science,0.544850,6447,"[0.4215032589683878, 0.5228824206263172, 0.616..."
5,electric_engineering_and_systems_science,0.545318,1173,"[0.7714343585409058, 0.5515016707082838, 0.458..."
6,quantitative_finance,0.566968,91,"[0.6739212881778495, 0.5768704940908814, 0.525..."
7,physics,0.574988,3259,"[0.583989488161111, 0.583989488161111, 0.59629..."


Length of data: 1114


,id,title_levenshtein,abstract_levenshtein,authors_levenshtein
0,2305.00604,0.0,0.0,22.285714
1,2305.00605,0.0,0.0,25.333333
2,2305.00607,0.0,55.0,16.666667
3,2305.00616,0.0,0.0,22.500000
4,2305.00617,0.0,0.0,25.000000
...,...,...,...,...
1120,2305.03231,0.0,0.0,30.428571
1121,2305.03233,0.0,0.0,28.571429
1122,2305.03234,0.0,0.0,21.333333
1123,2305.03235,0.0,0.0,21.714286


In [3]:
df=df1[['title_levenshtein', 'abstract_levenshtein', 'authors_levenshtein']]
df=df.to_dict('list')
threshold=[[1, 200, 10], [5, 300, 20], [10, 400, 30]]
alphalevel=0.05
alpha=[1]*3
beta=[1]*3
result = Bayes(df, threshold, alpha, beta, alphalevel)
display(result.res)
result.Plot_Bayes(yrange=[0,1.1, 0.2], height=600)

,Component,alpha,beta,Expected Success Probability,Variance of Success Probability,95.0%-level Credible Interval,Threshold
0,title_levenshtein,7042,184,0.9745,0.000003,"[0.9708, 0.978]","[1, 200, 10]"
1,abstract_levenshtein,4377,2849,0.6057,0.000033,"[0.5944, 0.617]","[1, 200, 10]"
2,authors_levenshtein,34,7192,0.0047,0.000001,"[0.0033, 0.0064]","[1, 200, 10]"
0,title_levenshtein,7083,143,0.9802,0.000003,"[0.9769, 0.9833]","[5, 300, 20]"
1,abstract_levenshtein,4876,2350,0.6748,0.000030,"[0.6639, 0.6855]","[5, 300, 20]"
2,authors_levenshtein,2681,4545,0.3710,0.000032,"[0.3599, 0.3822]","[5, 300, 20]"
0,title_levenshtein,7115,111,0.9846,0.000002,"[0.9817, 0.9873]","[10, 400, 30]"
1,abstract_levenshtein,5396,1830,0.7467,0.000026,"[0.7367, 0.7567]","[10, 400, 30]"
2,authors_levenshtein,6626,600,0.9170,0.000011,"[0.9105, 0.9232]","[10, 400, 30]"


In [4]:
df=[]
threshold=[[0.2]*8, [0.4]*8, [0.6]*8, [0.8]*8]
alphalevel=0.05
alpha=[1]*8
beta=[1]*8

for i in range(len(df2)):
    df.append( (df2['Name'][i], df2['Cosim_Values'][i]) )
df=dict(df)

result = Bayes(df, threshold, alpha, beta, alphalevel, direction="greater")
display(result.res)
result.Plot_Bayes(yrange=[-0.01,1.01,0.2], height=1000)

,Component,alpha,beta,Expected Success Probability,Variance of Success Probability,95.0%-level Credible Interval,Threshold
0,mathematics,1601,127,0.9265,0.000039,"[0.9137, 0.9383]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
1,economics,101,11,0.9018,0.000784,"[0.8406, 0.9495]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
2,quantitative_biology,193,4,0.9797,0.000100,"[0.9559, 0.9944]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
3,statistics,585,20,0.9669,0.000053,"[0.9513, 0.9797]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
4,computer_science,6297,152,0.9764,0.000004,"[0.9726, 0.98]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
5,electric_engineering_and_systems_science,1133,42,0.9643,0.000029,"[0.9529, 0.9741]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
6,quantitative_finance,90,3,0.9677,0.000332,"[0.9237, 0.9932]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
7,physics,3141,120,0.9632,0.000011,"[0.9565, 0.9694]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
0,mathematics,1470,258,0.8507,0.000073,"[0.8335, 0.8671]","[0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]"
1,economics,95,17,0.8482,0.001139,"[0.7765, 0.9082]","[0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]"


In [5]:
total = []
for i in df2['Cosim_Values']:
    total += i
total=dict(Value=total)
threshold=[[0.2],[0.4],[0.6],[0.8]]
alphalevel=0.05
alpha=[1]
beta=[1]

result=Bayes(total, threshold, alpha, beta, alphalevel, direction='greater')
result.res
result.Plot_Bayes(yrange=[0,1.01,0.2], height=800)


In [6]:
#use the total result as prior information to re-estimate p by categories. Got smaller CIs.
df=[]
threshold=[[0.2]*8, [0.5]*8, [0.8]*8, [0.9]*8]
alphalevel=0.05
alpha=[35838/625]*8
beta=[13937/635]*8

for i in range(len(df2)):
    df.append( (df2['Name'][i], df2['Cosim_Values'][i]) )
df=dict(df)

result = Bayes(df, threshold, alpha, beta, alphalevel, direction="greater")
display(result.res)
result.Plot_Bayes(yrange=[0,1.01,0.1], height=600)

,Component,alpha,beta,Expected Success Probability,Variance of Success Probability,95.0%-level Credible Interval,Threshold
0,mathematics,1657.3408,147.948031,0.9180,0.000042,"[0.905, 0.9303]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
1,economics,157.3408,31.948031,0.8312,0.000737,"[0.7748, 0.881]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
2,quantitative_biology,249.3408,24.948031,0.9090,0.000300,"[0.8724, 0.9401]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
3,statistics,641.3408,40.948031,0.9400,0.000083,"[0.921, 0.9565]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
4,computer_science,6353.3408,172.948031,0.9735,0.000004,"[0.9695, 0.9773]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
5,electric_engineering_and_systems_science,1189.3408,62.948031,0.9497,0.000038,"[0.937, 0.9611]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
6,quantitative_finance,146.3408,23.948031,0.8594,0.000706,"[0.8035, 0.9073]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
7,physics,3197.3408,140.948031,0.9578,0.000012,"[0.9507, 0.9643]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
0,mathematics,1237.3408,567.948031,0.6854,0.000119,"[0.6638, 0.7066]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]"
1,economics,135.3408,53.948031,0.7150,0.001071,"[0.6488, 0.7769]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]"


In [7]:
df=df1_gpt4[['title_levenshtein', 'abstract_levenshtein', 'authors_levenshtein']]
df=df.to_dict('list')
threshold=[[1, 200, 10], [5, 300, 20], [10, 400, 30]]
alphalevel=0.05
alpha=[1]*3
beta=[1]*3
result = Bayes(df, threshold, alpha, beta, alphalevel)
display(result.res)
result.Plot_Bayes(yrange=[0,1.1, 0.2], height=600)

,Component,alpha,beta,Expected Success Probability,Variance of Success Probability,95.0%-level Credible Interval,Threshold
0,title_levenshtein,1096,20,0.9821,0.000016,"[0.9735, 0.989]","[1, 200, 10]"
1,abstract_levenshtein,1073,43,0.9615,0.000033,"[0.9494, 0.972]","[1, 200, 10]"
2,authors_levenshtein,6,1110,0.0054,0.000005,"[0.002, 0.0104]","[1, 200, 10]"
0,title_levenshtein,1100,16,0.9857,0.000013,"[0.9779, 0.9918]","[5, 300, 20]"
1,abstract_levenshtein,1081,35,0.9686,0.000027,"[0.9576, 0.978]","[5, 300, 20]"
2,authors_levenshtein,350,766,0.3136,0.000193,"[0.2867, 0.3411]","[5, 300, 20]"
0,title_levenshtein,1104,12,0.9892,0.000010,"[0.9824, 0.9944]","[10, 400, 30]"
1,abstract_levenshtein,1091,25,0.9776,0.000020,"[0.9681, 0.9854]","[10, 400, 30]"
2,authors_levenshtein,1010,106,0.9050,0.000077,"[0.8872, 0.9215]","[10, 400, 30]"


### Section 4: Monte Carlo Estimation of Type I Error 

In [6]:
#Calculate the probability that p1 <= p2 using Monte Carlo method
n=10000
a1=2
a2=1096
b1=2
b2=20

p1=stats.beta.rvs(a1, b1, size=n)
p2=stats.beta.rvs(a2, b2, size=n)

res=dict(X=p1, Y=p2, Int=[])
res['Int'] = [1 if i<=j else 0 for i,j in zip(p1, p2)]
print("Probability: " + str(np.average(res['Int'])*100)+'%')

res=dict(X=p1, Y=p2, Z=stats.beta.pdf(p1, a1, b1)*stats.beta.pdf(p2, a2, b2))
fig=px.scatter_3d(res, x="X", y="Y", z="Z", width=1300, height=1200)
fig.update_traces(marker_size=1)
fig.show()

Probability: 99.9%
